### Wrinkle Detection
#### U-net w. f-1 loss
* https://datalab.snu.ac.kr/datalab-internal/gpu-status/
* warhol2
  * `jupyter lab --ip=147.46.216.82 --NotebookApp.password='sha1:6d8bb616ac21:dc1b7ebffd85cb159379a282c6b49e6121e0ffb1'`

In [1]:
import torch
print(torch.cuda.device(0))
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())
# print(torch.cuda.memory_allocated())
# print(torch.cuda.memoy_cached())

4
GeForce GTX 1080 Ti
True


In [2]:
import argparse

import os
import csv
import datetime
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from unet.model import UNet
from unet.dataset import *
from unet.util import *
from unet.train import train
from unet.evaluate import evaluate
from unet.loss import f1_loss, weighted_loss_and_f1_loss

import matplotlib.pyplot as plt

from torchvision import transforms, datasets

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [7]:
def print_log(key, lr, batch_size, num_epoch, loss_name, data_dir, ckpt_dir, log_dir, mode, device, train_continue):
    time = datetime.datetime.today()
    time = time.strftime('%m-%d_%H-%M')
    time_and_key_and_loss = time + "_" + key + '_'+ loss_name
    if ckpt_dir == "./checkpoint":
        ckpt_dir = os.path.join(ckpt_dir, time_and_key_and_loss)
    log_dir = os.path.join(log_dir, time_and_key_and_loss)
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
        
    print(f'tensorboard --logdir {log_dir} --host "147.46.216.169" --port 6006')
    
    # log parameters
    print("learning rate: %.4e" % lr)
    print("batch size: %d" % batch_size)
    print("number of epoch: %d" % num_epoch)
    print("loss function : %s" % loss_name)
    print("data dir: %s" % data_dir)
    print("ckpt dir: %s" % ckpt_dir)
    print("log dir: %s" % log_dir)
    print("mode: %s" % mode)
    print("device: %s" % device)
    print("train_continue: %s" % train_continue)
    f = open(os.path.join(log_dir, 'parameter.txt'), 'w')
    f.write("learning rate: %.4e\n" % lr)
    f.write("batch size: %d\n" % batch_size)
    f.write("number of epoch: %d\n" % num_epoch)
    f.write("loss function : %s\n" % loss_name)
    f.write("data dir: %s\n" % data_dir)
    f.write("ckpt dir: %s\n" % ckpt_dir)
    f.write("log dir: %s\n" % log_dir)
    f.write("mode: %s\n" % mode)
    f.write("device: %s\n" % device)
    f.write("train_continue: %s\n" % train_continue)
    f.close()
    return ckpt_dir, log_dir

In [5]:
def train_unet(data_dir, loss_name='weighted_BCE', lr=1e-3, batch_size=8, num_epoch=300, ckpt_dir="./checkpoint", log_dir="./log", mode="train", train_continue="off", key='eye_left'):
    
    train_transform = transforms.Compose([RandomResizedCrop(ratio=0.3), Normalization(mean=0.5, std=0.5), RandomFlip(), ToTensor()])
    val_transform = transforms.Compose([Normalization(mean=0.5, std=0.5), ToTensor()])

    dataset_train = Dataset(data_dir=os.path.join(data_dir, 'train'), transform=train_transform)
    loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True, num_workers=8)

    dataset_val = Dataset(data_dir=os.path.join(data_dir, 'val'), transform=val_transform)
    loader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=False, num_workers=8)

    num_data_train = len(dataset_train)
    num_data_val = len(dataset_val)

    num_batch_train = np.ceil(num_data_train / batch_size)
    num_batch_val = np.ceil(num_data_val / batch_size)

    
    if loss_name == "BCE":
        loss_function = nn.BCEWithLogitsLoss().to(device)
    elif loss_name == "weighted_BCE":
        loss_function = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([5])).to(device)  # 237 - > 59 -> 15
    elif loss_name == "f1":
        loss_function = f1_loss
    elif loss_name =="mix":
        mix_class = weighted_loss_and_f1_loss(nn.BCEWithLogitsLoss(pos_weight=torch.tensor([15])).to(device))
        loss_function = mix_class.loss
    else:
        assert False, loss_name + " is not supported"
        
    ckpt_dir, log_dir = print_log(key, lr, batch_size, num_epoch, loss_name, data_dir, ckpt_dir, log_dir, mode, device, train_continue)
        
    net = UNet()
    if torch.cuda.device_count() > 1:
        net = nn.DataParallel(net)
    net.to(device)
    
    optim = torch.optim.Adam(net.parameters(), lr=lr)

    writer_train = SummaryWriter(log_dir=os.path.join(log_dir, 'train'))
    writer_val = SummaryWriter(log_dir=os.path.join(log_dir, 'val'))

    train_f = open(os.path.join(log_dir, 'train.tsv'), 'w', encoding='utf-8', newline='')
    train_wr = csv.writer(train_f, delimiter='\t')
    val_f = open(os.path.join(log_dir, 'val.tsv'), 'w', encoding='utf-8', newline='')
    val_wr = csv.writer(val_f, delimiter='\t')
    train_wr.writerow(["#epoch/loss/acc/f1/tp/tn/fp/fn"])
    val_wr.writerow(["#epoch/loss/acc/f1/tp/tn/fp/fn"])

    ## train network
    st_epoch = 0

    if train_continue == "on":
        net, optim, st_epoch = load(ckpt_dir=ckpt_dir, net=net, optim=optim)

    for epoch in range(st_epoch, st_epoch + num_epoch):
        net.train()
        loss, acc, f1, tp, tn, fp, fn = train(net, loader_train, loss_function, num_batch_train, epoch, writer_train, device, optim)
        train_wr.writerow((epoch, loss, acc, f1, tp, tn, fp, fn))

        loss, acc, f1, tp, tn, fp, fn = evaluate(net, loader_val, loss_function, num_batch_val, epoch, writer_val, device)
        val_wr.writerow((epoch, loss, acc, f1, tp, tn, fp, fn))

        if epoch+1 == num_epoch: #((epoch+1) % (num_epoch / 10)) == 0:
            save(ckpt_dir=ckpt_dir, net=net, optim=optim, epoch=epoch+1)
        print("------------------------------------------------------------")

    writer_train.close()
    writer_val.close()
    train_f.close()
    val_f.close()
    
    return ckpt_dir, log_dir

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from numpy.random import randn

def plot_f1(logs):
    markers = ("o", "x", "s", "^")
    colors = ('dodgerblue','mediumseagreen', 'hotpink', '#fba84a')
    
    def plot_train():
        plt.rcParams["figure.figsize"] = (16,4)
        plt.rcParams['lines.linewidth'] = 2
        plt.rcParams['lines.color'] = 'r'
        plt.rcParams['axes.grid'] = True
        plt.rcParams['axes.spines.right'] = False
        plt.rcParams['axes.spines.top'] = False

        plt.suptitle('F1 Score - Train', fontsize=20)

        for i, k in enumerate(logs):
            # epoch/loss/acc/f1/tp/tn/fp/fn
            df = pd.read_csv(f"{logs[k]}/val.tsv", delimiter='\t', header=None, skiprows=1)
            val = df[3].to_numpy()
            df = pd.read_csv(f"{logs[k]}/train.tsv", delimiter='\t', header=None, skiprows=1)
            train = df[3].to_numpy()
            plt.plot(train, color=colors[i], marker=markers[i])
        plt.legend(logs.keys(), fontsize=15)
        plt.xlabel('epoch', fontsize=15)
        plt.ylabel('f1 score', fontsize=15)
        plt.show()
    def plot_val():
        plt.rcParams["figure.figsize"] = (16,4)
        plt.rcParams['lines.linewidth'] = 2
        plt.rcParams['lines.color'] = 'r'
        plt.rcParams['axes.grid'] = True
        plt.rcParams['axes.spines.right'] = False
        plt.rcParams['axes.spines.top'] = False

        plt.suptitle('F1 Score - Validation', fontsize=20)

        for i, k in enumerate(logs):
            # epoch/loss/acc/f1/tp/tn/fp/fn
            df = pd.read_csv(f"{logs[k]}/val.tsv", delimiter='\t', header=None, skiprows=1)
            val = df[3].to_numpy()
            df = pd.read_csv(f"{logs[k]}/train.tsv", delimiter='\t', header=None, skiprows=1)
            train = df[3].to_numpy()
            plt.plot(val, color=colors[i], marker=markers[i])
        plt.legend(logs.keys(), fontsize=15)
        plt.xlabel('epoch', fontsize=15)
        plt.ylabel('f1 score', fontsize=15)
        plt.show()
    plot_train()
    plot_val()
    
def plot_loss(logs):
    markers = ("o", "x", "s", "^")
    colors = ('dodgerblue','mediumseagreen', 'hotpink', '#fba84a')
    
    def plot_train():
        plt.rcParams["figure.figsize"] = (16,4)
        plt.rcParams['lines.linewidth'] = 2
        plt.rcParams['lines.color'] = 'r'
        plt.rcParams['axes.grid'] = True
        plt.rcParams['axes.spines.right'] = False
        plt.rcParams['axes.spines.top'] = False

        plt.suptitle('Loss - Train', fontsize=20)

        for i, k in enumerate(logs):
            # epoch/loss/acc/f1/tp/tn/fp/fn
            df = pd.read_csv(f"{logs[k]}/val.tsv", delimiter='\t', header=None, skiprows=1)
            val = df[1].to_numpy()
            df = pd.read_csv(f"{logs[k]}/train.tsv", delimiter='\t', header=None, skiprows=1)
            train = df[1].to_numpy()
            plt.plot(train, color=colors[i], marker=markers[i])
        plt.legend(logs.keys(), fontsize=15)
        plt.xlabel('epoch', fontsize=15)
        plt.ylabel('loss', fontsize=15)
        plt.show()
    def plot_val():
        plt.rcParams["figure.figsize"] = (16,4)
        plt.rcParams['lines.linewidth'] = 2
        plt.rcParams['lines.color'] = 'r'
        plt.rcParams['axes.grid'] = True
        plt.rcParams['axes.spines.right'] = False
        plt.rcParams['axes.spines.top'] = False

        plt.suptitle('Loss - Validation', fontsize=20)

        for i, k in enumerate(logs):
            # epoch/loss/acc/f1/tp/tn/fp/fn
            df = pd.read_csv(f"{logs[k]}/val.tsv", delimiter='\t', header=None, skiprows=1)
            val = df[1].to_numpy()
            df = pd.read_csv(f"{logs[k]}/train.tsv", delimiter='\t', header=None, skiprows=1)
            train = df[1].to_numpy()
            plt.plot(val, color=colors[i], marker=markers[i])
        plt.legend(logs.keys(), fontsize=15)
        plt.xlabel('epoch', fontsize=15)
        plt.ylabel('loss', fontsize=15)
        plt.show()
    plot_train()
    plot_val()

### U-net baseline
* `train_unet(data_dir, loss_name)`
  * Loss function: `BCE`, `weight_BCE`, `f1`, `mix`

In [8]:
logs = {}
ckpts = {}
new_dict = {
    'eye_left': (0, 575, 220, 603),
    'eye_right': (672, 1247, 220, 603),
    'nose_left': (350, 573, 400, 943),
    'nose_right': (700, 923, 400, 943),
    'mouth': (300, 971, 940, 1195),
    'forehead': (0, 1247, 0, 223),
    'cheek_right': (896, 1247, 576, 1343),
    'cheek_left': (0, 351, 576, 1343),
    'jaw': (350, 925, 1152, 1343),
    'center': (550, 709, 220, 955)
}

In [ ]:
for key in new_dict:
    new_logs = {}
    new_ckpts = {}
    for loss in ('BCE', 'weighted_BCE', 'f1', 'mix'):
        new_ckpts[loss], new_logs[loss] = train_unet(f"../../../split_data/{key}/", loss, num_epoch=200, batch_size=16, key=key)
    logs[key] = new_logs
    ckpts[key] = new_ckpts

tensorboard --logdir ./log/12-15_22-29_eye_left_BCE --host "147.46.216.169" --port 6006
learning rate: 1.0000e-03
batch size: 16
number of epoch: 200
loss function : BCE
data dir: ../../../split_data/eye_left/
ckpt dir: ./checkpoint/12-15_22-29_eye_left_BCE
log dir: ./log/12-15_22-29_eye_left_BCE
mode: train
device: cuda
train_continue: off
TRAIN: EPOCH 0000 | BATCH 0001 / 0028 | LOSS 0.6106 | ACC 0.9363 | F1 SCORE 0.0219 | TIME 12.4751
TRAIN: EPOCH 0000 | BATCH 0002 / 0028 | LOSS 0.5569 | ACC 0.9631 | F1 SCORE 0.0176 | TIME 12.9703
TRAIN: EPOCH 0000 | BATCH 0003 / 0028 | LOSS 0.4816 | ACC 0.9873 | F1 SCORE 0.0156 | TIME 13.4253
TRAIN: EPOCH 0000 | BATCH 0004 / 0028 | LOSS 0.4370 | ACC 0.9827 | F1 SCORE 0.0213 | TIME 13.8972
TRAIN: EPOCH 0000 | BATCH 0005 / 0028 | LOSS 0.4011 | ACC 0.9802 | F1 SCORE 0.0221 | TIME 14.3922
TRAIN: EPOCH 0000 | BATCH 0006 / 0028 | LOSS 0.3690 | ACC 0.9838 | F1 SCORE 0.0227 | TIME 15.1372
TRAIN: EPOCH 0000 | BATCH 0007 / 0028 | LOSS 0.3622 | ACC 0.9844 | F1

In [ ]:
eye_logs['BCE'] = "./log/11-30_17-26_BCE"
eye_logs['weighted_BCE'] = "./log/11-30_18-34_weighted_BCE"
eye_logs['f1'] = "./log/11-30_19-41_f1"
eye_logs['mix'] = "./log/11-30_20-49_mix"

plot_f1(eye_logs)

In [ ]:
plot_loss(eye_logs)